In [ ]:
# ==========================================
# PART 1: INSTALL, IMPORTS & SETTINGS
# ==========================================
!pip install -U -q gradio nltk pandas matplotlib requests google-generativeai google-api-python-client google-auth-httplib2 google-auth-oauthlib fpdf transformers torch pillow

import gradio as gr
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import requests
import json
import os
import re
from collections import defaultdict
from google.colab import auth
from googleapiclient.discovery import build
import google.generativeai as genai
from fpdf import FPDF
from transformers import pipeline
from PIL import Image as PILImage

# --- Configuration ---
# Set API Key
os.environ["GEMINI_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

# Constants
FOLDER_ID = '1BWa5Hy-4aTifH0zHULoPMwi9qL-s_5-l'
BASE_URL = "https://server-cloud-v645.onrender.com/"
MODEL_ID = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"

# NLTK Downloads
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

# Global Variables placeholders (will be filled in Main)
GLOBAL_CACHE = { "temperature": None, "humidity": None, "soil": None }
DOC_TITLES = {}
ARTICLES = {}
drive_service = None
plant_classifier = None
engine = None
ACTIVE_MODEL = "models/gemini-pro"

In [7]:
# ==========================================
# PART 2: HELPER FUNCTIONS & CLASSES
# ==========================================

# --- 1. Drive & File Helpers ---
def get_files_from_folder(folder_id):
    try:
        results = drive_service.files().list(
            q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.document' and trashed=false",
            fields="files(id, name)", pageSize=20
        ).execute()
        return results.get('files', [])
    except: return []

def fetch_gdoc_content(file_id):
    try:
        content = drive_service.files().export_media(fileId=file_id, mimeType='text/plain').execute()
        return content.decode('utf-8')
    except: return ""

# --- 2. Search Engine Logic (Class) ---
class LectureSearchEngine:
    def __init__(self):
        self.word_locations = defaultdict(list)
        self.documents = {}
        self.titles = {}
        self.stop_words = set(stopwords.words('english'))
        self.stop_words.update({'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to', 'for', 'of', 'with'})

    def build_index(self, docs, titles):
        self.documents = docs
        self.titles = titles
        self.word_locations.clear()
        for doc_id, content in self.documents.items():
            words = re.findall(r'\w+', content.lower())
            word_counts = defaultdict(int)
            for word in words:
                if word not in self.stop_words:
                    word_counts[word] += 1
            for word, count in word_counts.items():
                self.word_locations[word].append((doc_id, count))

    def get_context(self, content, query_words, window=150):
        content_lower = content.lower()
        best_idx = -1
        for word in query_words:
            idx = content_lower.find(word)
            if idx != -1:
                best_idx = idx
                break
        if best_idx != -1:
            start = max(0, best_idx - 50)
            end = min(len(content), best_idx + window)
            return "..." + content[start:end].replace("\n", " ") + "..."
        return content[:200] + "..."

    def search(self, query, num_results=3):
        query_words = [word.lower() for word in re.findall(r'\w+', query) if word.lower() not in self.stop_words]
        if not query_words: return []
        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})
        for word in query_words:
            for doc_id, freq in self.word_locations.get(word, []):
                page_scores[doc_id]['matches'] += 1
                page_scores[doc_id]['total_freq'] += freq
        ranked_results = [(doc_id, scores['matches'], scores['total_freq']) for doc_id, scores in page_scores.items()]
        ranked_results.sort(key=lambda x: (x[1], x[2]), reverse=True)
        results = []
        for doc_id, matches, total_freq in ranked_results[:num_results]:
            title = self.titles.get(doc_id, "Unknown")
            content = self.documents.get(doc_id, "")
            context = self.get_context(content, query_words)
            results.append({'title': title, 'score': f"Matches: {matches}, Freq: {total_freq}", 'context': context})
        return results

# --- 3. Gemini & RAG Helpers ---
def get_working_model():
    genai.configure(api_key=GEMINI_API_KEY)
    try:
        models = [m for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
        for m in models:
            if 'flash' in m.name.lower(): return m.name
        if models: return models[0].name
        return "models/gemini-pro"
    except: return "models/gemini-pro"

def search_engine_rag(query):
    if not ARTICLES: return "⚠️ Error: No documents loaded."
    results = engine.search(query)
    if not results: return f"No results found for: '{query}'"
    output_log = f"🔎 Found {len(results)} docs (Ranked by Matches & Freq)\n" + "="*40 + "\n"
    context_text = []
    for res in results:
        output_log += f"\n📄 [{res['title']}] ({res['score']})\n - {res['context']}\n"
        context_text.append(f"Source ({res['title']}): {res['context']}")
    try:
        model = genai.GenerativeModel(ACTIVE_MODEL)
        prompt = (f"Question: {query}\nBase your answer ONLY on the following context:\n" + "\n".join(context_text))
        response = model.generate_content(prompt)
        gemini_summary = f"\n🤖 AI Answer:\n{response.text}\n"
    except Exception as e: gemini_summary = f"\n(AI Error: {e})\n"
    return gemini_summary + "\n" + output_log

def get_index_table():
    data = []
    if engine:
        for i, (word, locs) in enumerate(engine.word_locations.items()):
            if i >= 100: break
            data.append({"term": word, "docs": str(locs)})
    return pd.DataFrame(data)

# --- 4. IoT Helpers ---
def fetch_data_as_df(feed, limit):
    try:
        resp = requests.get(f"{BASE_URL}/history", params={"feed": feed, "limit": limit}, timeout=5)
        data = resp.json()
        if "data" in data and len(data["data"]) > 0:
            df = pd.DataFrame(data["data"])
            df["created_at"] = pd.to_datetime(df["created_at"])
            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df = df.sort_values("created_at")
            GLOBAL_CACHE[feed] = df
            return df
    except: return None
    return None

def create_plot(df, title, color):
    if df is None or df.empty: return None
    fig, ax = plt.subplots(figsize=(8, 3.5))
    ax.plot(df["created_at"], df["value"], marker='.', linestyle='-', color=color, linewidth=1.5)
    ax.set_title(title)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

def update_iot_view(chk_temp, chk_hum, chk_soil, chk_json, limit):
    fig_temp, fig_hum, fig_soil, json_file = None, None, None, None
    log_messages = []
    export_data = {}
    if chk_temp:
        df = fetch_data_as_df("temperature", limit)
        if df is not None:
            fig_temp = create_plot(df, "Temp", "red")
            export_data["temperature"] = df.to_dict(orient="records")
    if chk_hum:
        df = fetch_data_as_df("humidity", limit)
        if df is not None:
            fig_hum = create_plot(df, "Hum", "blue")
            export_data["humidity"] = df.to_dict(orient="records")
    if chk_soil:
        df = fetch_data_as_df("soil", limit)
        if df is not None:
            fig_soil = create_plot(df, "Soil", "brown")
            export_data["soil"] = df.to_dict(orient="records")
    if chk_json:
        json_file = "iot_data.json"
        with open(json_file, 'w') as f: json.dump(export_data, f, default=str)
        log_messages.append("JSON Saved!")
    return fig_temp, fig_hum, fig_soil, json_file, "\n".join(log_messages)

def refresh_dashboard_real():
    limit = 20
    df_t = fetch_data_as_df("temperature", limit)
    df_h = fetch_data_as_df("humidity", limit)
    df_s = fetch_data_as_df("soil", limit)
    fig_t = create_plot(df_t, "Temp Trend", "#ff6b6b")
    fig_h = create_plot(df_h, "Hum Trend", "#4ecdc4")
    fig_s = create_plot(df_s, "Soil Trend", "#8d6e63")
    val_t = df_t.iloc[-1]["value"] if df_t is not None else 0
    val_h = df_h.iloc[-1]["value"] if df_h is not None else 0
    val_s = df_s.iloc[-1]["value"] if df_s is not None else 0
    status = "Warning ⚠️" if val_t > 35 else "OK ✅"
    return fig_t, fig_h, fig_s, f"{val_t} °C", f"{val_h} %", f"{val_s}", status

# --- 5. Image AI Helpers ---
def analyze_image(img):
    if img is None: return "⚠️ Please upload an image."
    if plant_classifier is None: return "❌ Model not loaded."
    try:
        raw_image = PILImage.fromarray(img.astype('uint8'), 'RGB')
        results = plant_classifier(raw_image)
        top_result = results[0]
        label = top_result['label']
        score = top_result['score']
        if "healthy" in label.lower(): return f"✅ Healthy ({label})\nConfidence: {score:.1%}"
        else: return f"⚠️ Potential Issue: {label.replace('_', ' ').title()}\nConfidence: {score:.1%}"
    except Exception as e: return f"❌ Error: {str(e)}"

# --- 6. PDF Report Helpers ---
class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 15)
        self.cell(0, 10, 'IoT System Report', 0, 1, 'C')
        self.ln(5)
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.set_fill_color(200, 220, 255)
        self.cell(0, 10, title, 0, 1, 'L', 1)
        self.ln(4)
    def chapter_body(self, df):
        self.set_font('Arial', '', 10)
        self.cell(90, 8, 'Timestamp', 1)
        self.cell(40, 8, 'Value', 1)
        self.ln()
        if df is not None and not df.empty:
            for index, row in df.sort_values("created_at", ascending=False).head(50).iterrows():
                self.cell(90, 8, str(row['created_at']), 1)
                self.cell(40, 8, str(row['value']), 1)
                self.ln()
        self.ln(10)

def generate_pdf_report():
    pdf = PDFReport()
    pdf.add_page()
    for feed in ["temperature", "humidity", "soil"]:
        df = GLOBAL_CACHE.get(feed)
        if df is None: df = fetch_data_as_df(feed, 100)
        pdf.chapter_title(f"{feed.capitalize()} Data")
        pdf.chapter_body(df)
    filename = "iot_report.pdf"
    pdf.output(filename)
    return filename, f"✅ Report Saved to {filename}"

In [ ]:
# ==========================================
# PART 3: MAIN EXECUTION
# ==========================================

print("🔄 Connecting to Google Drive... Please approve.")
try:
    auth.authenticate_user()
    drive_service = build('drive', 'v3')
    print("✅ Drive Connected!")
except Exception as e:
    print(f"❌ Drive Error: {e}")

# 1. Load Files & Build Index
files = get_files_from_folder(FOLDER_ID)
if files:
    print(f"📂 Found {len(files)} documents.")
    for i, file in enumerate(files, 1):
        doc_id = str(i)
        DOC_TITLES[doc_id] = file['name']
        content = fetch_gdoc_content(file['id']).strip()
        if content: ARTICLES[doc_id] = content
else:
    print("⚠️ No documents found or wrong Folder ID.")

engine = LectureSearchEngine()
if ARTICLES:
    print("📚 Building Search Index...")
    engine.build_index(ARTICLES, DOC_TITLES)

# 2. Setup AI Models
ACTIVE_MODEL = get_working_model()
print(f"🤖 Active Text AI: {ACTIVE_MODEL}")

print(f"🔄 Loading Vision Model ({MODEL_ID})...")
try:
    plant_classifier = pipeline("image-classification", model=MODEL_ID)
    print("✅ Vision Model Loaded!")
except:
    print("❌ Vision Model Failed to Load.")
    plant_classifier = None

# 3. Launch Gradio UI
print("\n🚀 Launching System...")

# JavaScript for Light/Dark Mode Toggle
js_toggle = """
function toggleTheme() {
    const body = document.querySelector('body');
    if (body.classList.contains('dark')) {
        body.classList.remove('dark');
    } else {
        body.classList.add('dark');
    }
}
"""

# Custom Green Theme
theme = gr.themes.Soft(
    primary_hue="green",
    secondary_hue="emerald",
).set(
    body_background_fill="*neutral_50",
    block_background_fill="*neutral_100"
)

with gr.Blocks(theme=theme, title="Smart Plant System", js=js_toggle) as demo:

    # Header Section
    with gr.Row():
        with gr.Column(scale=5):
            gr.Markdown("# 🌱 Smart Plant System Ultimate")
        with gr.Column(scale=1):
            # Toggle Button
            mode_btn = gr.Button("🌗 Light/Dark Mode", variant="secondary")
            mode_btn.click(None, None, None, js="toggleTheme")

    with gr.Tabs():

        # Tab 1: Image AI
        with gr.TabItem("1. Image (AI) 📸"):
            gr.Markdown("### 🍃 Plant Disease Detection")
            with gr.Row():
                with gr.Column():
                    img_input = gr.Image(height=300, label="Upload Leaf Photo 📷", type="numpy")
                    analyze_btn = gr.Button("🔍 Analyze Leaf", variant="primary")
                with gr.Column():
                    img_out = gr.Textbox(label="AI Diagnosis 🤖", lines=4, placeholder="Waiting for image...")
            analyze_btn.click(analyze_image, inputs=img_input, outputs=img_out)

        # Tab 2: IoT Data
        with gr.TabItem("2. IoT Data 📊"):
            gr.Markdown("### 📡 Sensor History Analysis")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("**Select Sensors:**")
                    c1 = gr.Checkbox(label="Temperature 🌡️", value=True)
                    c2 = gr.Checkbox(label="Humidity 💧")
                    c3 = gr.Checkbox(label="Soil Moisture 🌿")
                    c4 = gr.Checkbox(label="Export JSON 💾")
                    lim = gr.Number(value=20, label="Data Limit 🔢")
                    btn = gr.Button("📥 Fetch Data", variant="primary")

                with gr.Column(scale=3):
                    p1 = gr.Plot(show_label=False)
                    p2 = gr.Plot(show_label=False)
                    p3 = gr.Plot(show_label=False)

            with gr.Row():
                f_out = gr.File(height=50, label="Download JSON")
                l_out = gr.Textbox(label="System Log 📝", lines=1)

            btn.click(update_iot_view, inputs=[c1,c2,c3,c4,lim], outputs=[p1,p2,p3,f_out,l_out])

        # Tab 3: Search Docs
        with gr.TabItem("3. Search Docs 🔍"):
            gr.Markdown("### 📚 Knowledge Base Search")
            with gr.Row():
                txt_in = gr.Textbox(label="Ask a Question", placeholder="How does photosynthesis work?...", scale=4)
                search_btn = gr.Button("🔎 Search", variant="primary", scale=1)

            res_out = gr.Textbox(label="AI & Doc Results 💡", lines=12)
            search_btn.click(search_engine_rag, inputs=txt_in, outputs=res_out)

            with gr.Accordion("🛠️ Index Debug View", open=False):
                gr.Dataframe(get_index_table)

        # Tab 4: Dashboard
        with gr.TabItem("4. Dashboard 🎛️"):
            gr.Markdown("### ⚡ Live Monitor")
            dash_btn = gr.Button("🔄 Sync Live Data", variant="primary")

            # Live Metrics
            with gr.Row():
                b1 = gr.Textbox(label="Temp 🌡️")
                b2 = gr.Textbox(label="Humidity 💧")
                b3 = gr.Textbox(label="Soil 🌿")
                b4 = gr.Textbox(label="Status 🚦")

            # Live Charts
            with gr.Row():
                dp1 = gr.Plot(label="Temp")
                dp2 = gr.Plot(label="Humidity")
            with gr.Row():
                dp3 = gr.Plot(label="Soil")

            dash_btn.click(refresh_dashboard_real, outputs=[dp1,dp2,dp3,b1,b2,b3,b4])

        # Tab 5: Report
        with gr.TabItem("5. PDF Report 📑"):
            gr.Markdown("### 📄 Generate Summary Report")
            report_btn = gr.Button("🖨️ Generate PDF", variant="primary")
            with gr.Row():
                pdf_file = gr.File(label="Download PDF 📥")
                report_log = gr.Textbox(label="Log 📝", lines=2)
            report_btn.click(generate_pdf_report, outputs=[pdf_file, report_log])

demo.launch(inline=True, height=900, debug=True)

🔄 Connecting to Google Drive... Please approve.


✅ Drive Connected!


📂 Found 5 documents.
📚 Building Search Index...
🤖 Active Text AI: models/gemini-2.5-flash
🔄 Loading Vision Model (linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification)...


/usr/local/lib/python3.12/dist-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:30: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(
Device set to use cpu
/tmp/ipython-input-3289238560.py:66: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, js. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=theme, title="Smart Plant System", js=js_toggle) as demo:


✅ Vision Model Loaded!

🚀 Launching System...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://51ae4a979923898df7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
